# Customizing Model.fit()

## 1. Imports and Configuration

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Configure GPU memory growth to be dynamic instead of allocating all memory at once
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

## 2. Data Loading and Preprocessing

## 3. Model Definition

## 4. Compile Model

## 5. Model Training and Evaluation